In [2]:
%pip install kaiwu-1.0.3-py3-none-any.whl
%pip install sklearn
%pip install pyqubo
%pip install dwave.system
%pip install scipy

Processing ./kaiwu-1.0.3-py3-none-any.whl
kaiwu is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated 

In [3]:
import kaiwu as kw
kw.license.init(user_id="72317291601100802", sdk_code="vDSsMrcS1XvoHxrKEyWGPu3y6bydtx")
from kaiwu.classical import SimulatedAnnealingOptimizer
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error
import random


Baseline models: linear/poly

In [4]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error

# Demand data (monthly time series)
demand = np.array([9000, 9400, 9594, 9859, 9958, 10043, 10309, 10512, 10588])

# Prepare input (X) and output (y) for training
# X will consist of lagged values (autoregressive approach)
lags = 2  # Use the last two months to predict the next month
X = np.array([demand[i:i + lags] for i in range(len(demand) - lags)])
y = demand[lags:]  # Target variable is the next month's demand

# Train/Test split
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Polynomial regression for more flexibility
degree = 2  # Degree of the polynomial
poly = PolynomialFeatures(degree)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Train the model
model = LinearRegression()
model.fit(X_train_poly, y_train)

# Predict on test data
y_pred = model.predict(X_test_poly)

# Calculate error metrics
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error on Test Data: {mse:.2f}")

# Predict the next value (October demand)
latest_lag = demand[-lags:]  # Use the last two months as input
next_input = poly.transform([latest_lag])  # Transform using PolynomialFeatures
predicted_y = model.predict(next_input)

print(f"Predicted Demand for October: {predicted_y[0]:.2f}")


Mean Squared Error on Test Data: 5418084.88
Predicted Demand for October: 14290.38


Baseline models: ar_model

In [5]:
# Historical demand data (January to September)
demand = np.array([9000, 9400, 9594, 9859, 9958, 10043, 10309, 10512, 10588])

# Define AR(2) model parameters (to be optimized)
def ar_model(params, demand):
    c, phi1, phi2 = params
    predicted = []
    for t in range(2, len(demand)):
        y_t = c + phi1 * demand[t - 1] + phi2 * demand[t - 2]
        predicted.append(y_t)
    return np.array(predicted)

# Objective function: minimize squared error
def objective(params, demand):
    c, phi1, phi2 = params
    predicted = ar_model(params, demand)
    observed = demand[2:]  # starting from index 2 due to AR(2) model
    error = np.sum((predicted - observed) ** 2)
    return error

# Initial guess for parameters [c, phi1, phi2]
initial_guess = [0, 0, 0]

# Solve using a classical optimizer (to later transform into QUBO logic)
result = minimize(objective, initial_guess, args=(demand,), method='Nelder-Mead')
c_opt, phi1_opt, phi2_opt = result.x

# Predict demand for October using the optimized AR(2) model
october_demand = c_opt + phi1_opt * demand[-1] + phi2_opt * demand[-2]

c_opt, phi1_opt, phi2_opt, october_demand


(-1.3954764576209415,
 1.052336371794449,
 -0.03604914801604513,
 10761.793384157338)

In [6]:
# Calculate Mean Squared Error (MSE) for the optimized AR(2) model
def calculate_mse(params, demand):
    predicted = ar_model(params, demand)
    observed = demand[2:]  # observed values starting from index 2
    mse = np.mean((predicted - observed) ** 2)
    return mse

# Compute MSE for the optimized parameters
mse = calculate_mse([c_opt, phi1_opt, phi2_opt], demand)
mse

6240.063117704558

Kaiwu Model

In [20]:
# Define the input data
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'June', 'Jul', 'Aug', 'Sept']
demands = np.array([9000, 9400, 9594, 9859, 9958, 10043, 10309, 10512, 10588])

# Calculate first differences to make the series more stationary
diff_demands = np.diff(demands)

# Function to convert binary variables to a floating-point value
def binary_to_float(binary_vars, min_val=-1, max_val=1):
    """Convert binary variables to float in range [min_val, max_val]"""
    weights = 2.0 ** -np.arange(1, len(binary_vars) + 1)
    return min_val + (max_val - min_val) * np.sum(weights * binary_vars)

# Define grid search space for Simulated Annealing parameters
param_grid = {
    'initial_temperature': [10, 30, 50, 100, 200, 400],
    'alpha': [0.8, 0.85, 0.9, 0.95, 0.99, 0.999],
    'iterations_per_t': [10, 15, 20, 50, 100],
    'cut_off': [0.1, 0.01, 0.001, 0.0001, 1e-6],
}

# Store the best solution
best_mse = float('inf')
best_params = {}
best_phi1 = 0
best_phi2 = 0

# Grid search over all combinations
for initial_temp in param_grid['initial_temperature']:
    for cooling_rate in param_grid['alpha']:
        for iterations_per_t in param_grid['iterations_per_t']:
            for cut_off in param_grid['cut_off']:
                # Create random initial coefficients for φ₁ and φ₂
                phi1 = random.uniform(-1, 1)
                phi2 = random.uniform(-1, 1)

                # Simulated Annealing
                temperature = initial_temp
                current_mse = float('inf')

                for _ in range(iterations_per_t):
                    # Perturb coefficients to create a neighbor solution
                    new_phi1 = phi1 + random.uniform(-0.01, 0.01)
                    new_phi2 = phi2 + random.uniform(-0.01, 0.01)

                    # Calculate new MSE
                    squared_errors = []
                    for t in range(2, len(diff_demands)):
                        predicted_diff = new_phi1 * diff_demands[t - 1] + new_phi2 * diff_demands[t - 2]
                        actual_diff = diff_demands[t]
                        squared_errors.append((predicted_diff - actual_diff) ** 2)
                    new_mse = np.mean(squared_errors)

                    # Accept solution based on Simulated Annealing criteria
                    delta_mse = new_mse - current_mse
                    if delta_mse < 0 or random.uniform(0, 1) < np.exp(-delta_mse / temperature):
                        phi1, phi2 = new_phi1, new_phi2
                        current_mse = new_mse

                    # Cool down temperature
                    temperature *= cooling_rate

                    # Stop if temperature is below the cutoff
                    if temperature < cut_off:
                        break

                # Update best solution if the current one is better
                if current_mse < best_mse:
                    best_mse = current_mse
                    best_params = {
                        'initial_temperature': initial_temp,
                        'alpha': cooling_rate,
                        'iterations_per_t': iterations_per_t,
                        'cut_off': cut_off,
                    }
                    best_phi1, best_phi2 = phi1, phi2

# Use the best AR(2) coefficients to predict October demand
last_diff = demands[-1] - demands[-2]
second_last_diff = demands[-2] - demands[-3]
predicted_diff = best_phi1 * last_diff + best_phi2 * second_last_diff
october_demand = demands[-1] + predicted_diff

# Print the best parameters, MSE, and predicted October demand
print(f"Best Parameters: {best_params}")
print(f"AR(2) Coefficients: φ₁ = {best_phi1:.4f}, φ₂ = {best_phi2:.4f}")
print(f"Best Mean Squared Error (MSE): {best_mse:.4f}")
print(f"Predicted demand for October: {october_demand:.0f}")


Best Parameters: {'initial_temperature': 10, 'alpha': 0.9, 'iterations_per_t': 100, 'cut_off': 0.001}
AR(2) Coefficients: φ₁ = 0.4742, φ₂ = 0.2744
Best Mean Squared Error (MSE): 10426.7537
Predicted demand for October: 10680


In [21]:
import numpy as np
import random

# Define the input data
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'June', 'Jul', 'Aug', 'Sept']
demands = np.array([9000, 9400, 9594, 9859, 9958, 10043, 10309, 10512, 10588])

# Calculate first differences to make the series more stationary
diff_demands = np.diff(demands)

# Normalize the differences for better optimization
mean_diff = np.mean(diff_demands)
std_diff = np.std(diff_demands)
normalized_diff = (diff_demands - mean_diff) / std_diff

# Store the best solution
best_mse = float('inf')
best_params = {}
best_phi1 = 0
best_phi2 = 0

# Define grid search space for Simulated Annealing parameters
param_grid = {
    'initial_temperature': [10, 30, 50, 100, 200, 400],
    'alpha': [0.8, 0.85, 0.9, 0.95, 0.99, 0.999],
    'iterations_per_t': [10, 20, 50, 100],
    'cut_off': [0.001, 0.0001, 1e-6],
}

# Grid search over all combinations
for initial_temp in param_grid['initial_temperature']:
    for cooling_rate in param_grid['alpha']:
        for iterations_per_t in param_grid['iterations_per_t']:
            for cut_off in param_grid['cut_off']:
                # Initialize φ₁ and φ₂ with small random values
                phi1 = random.uniform(-0.5, 0.5)
                phi2 = random.uniform(-0.5, 0.5)

                # Simulated Annealing
                temperature = initial_temp
                current_mse = float('inf')

                for _ in range(iterations_per_t):
                    # Perturb coefficients to create a neighbor solution
                    new_phi1 = phi1 + random.uniform(-0.005, 0.005)
                    new_phi2 = phi2 + random.uniform(-0.005, 0.005)

                    # Calculate new MSE
                    squared_errors = []
                    for t in range(2, len(normalized_diff)):
                        predicted_diff = new_phi1 * normalized_diff[t - 1] + new_phi2 * normalized_diff[t - 2]
                        actual_diff = normalized_diff[t]
                        squared_errors.append((predicted_diff - actual_diff) ** 2)
                    new_mse = np.mean(squared_errors)

                    # Penalize coefficients outside reasonable bounds
                    regularization_penalty = 0.1 * (new_phi1**2 + new_phi2**2)
                    new_mse += regularization_penalty

                    # Accept solution based on Simulated Annealing criteria
                    delta_mse = new_mse - current_mse
                    if delta_mse < 0 or random.uniform(0, 1) < np.exp(-delta_mse / temperature):
                        phi1, phi2 = new_phi1, new_phi2
                        current_mse = new_mse

                    # Cool down temperature
                    temperature *= cooling_rate

                    # Stop if temperature is below the cutoff
                    if temperature < cut_off:
                        break

                # Update best solution if the current one is better
                if current_mse < best_mse:
                    best_mse = current_mse
                    best_params = {
                        'initial_temperature': initial_temp,
                        'alpha': cooling_rate,
                        'iterations_per_t': iterations_per_t,
                        'cut_off': cut_off,
                    }
                    best_phi1, best_phi2 = phi1, phi2

# Rescale the inputs for prediction
last_diff = (demands[-1] - demands[-2] - mean_diff) / std_diff
second_last_diff = (demands[-2] - demands[-3] - mean_diff) / std_diff

# Predict demand for October
predicted_diff = best_phi1 * last_diff + best_phi2 * second_last_diff
predicted_diff = predicted_diff * std_diff + mean_diff
october_demand = demands[-1] + predicted_diff

# Print the best parameters, MSE, and predicted October demand
print(f"Best Parameters: {best_params}")
print(f"AR(2) Coefficients: φ₁ = {best_phi1:.4f}, φ₂ = {best_phi2:.4f}")
print(f"Best Mean Squared Error (MSE): {best_mse:.4f}")
print(f"Predicted demand for October: {october_demand:.0f}")


Best Parameters: {'initial_temperature': 100, 'alpha': 0.99, 'iterations_per_t': 10, 'cut_off': 1e-06}
AR(2) Coefficients: φ₁ = -0.1224, φ₂ = -0.1453
Best Mean Squared Error (MSE): 0.6918
Predicted demand for October: 10801
